In [13]:
import os
import getpass
from crewai import Agent, Task, Crew
from crewai_tools import PDFSearchTool
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key:  ········


In [14]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [16]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', google_api_key=SecretStr('**********'), temperature=1.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022A162C5700>, default_metadata=())

In [22]:

config = {
    "llm": {
        "provider": "google",
        "config": {
            "model": "gemini/gemini-pro",
        }
    },
    "embedder": {
        "provider": "google",
        "config": {
            "model": "models/embedding-001",
            "task_type": "retrieval_document",
        }
    },
}

pdf_tool = PDFSearchTool(
    pdf_path="./A_Dance_With_Dragons.pdf",
    config=config,
)

researcher = Agent(
    role="PDF Content Researcher",
    goal="Find specific information within a PDF document to answer questions.",
    backstory="A meticulous researcher specializing in PDF documents.",
    verbose=True,
    memory=True,
    tools=[pdf_tool],
    llm=llm,
)

task = Task(
    description="Answer the user's question using the information from the PDF.",
    agent=researcher,
    expected_output="An answer for the user's query"
)

crew = Crew(
    agents=[researcher],
    tasks=[task],
    verbose=True,
)

2024-12-19 08:02:02,894 - 13452 - llm.py-llm:187 - ERROR: Failed to get supported params: argument of type 'NoneType' is not iterable
2024-12-19 08:02:02,900 - 13452 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed
2024-12-19 08:02:02,904 - 13452 - llm.py-llm:187 - ERROR: Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



In [21]:
query = "How did Oberyn Martel die?"
result = crew.kickoff(inputs={
    "query": query,
})

print("\nResponse:")
print(result)

2024-12-19 08:00:40,766 - 13452 - llm.py-llm:187 - ERROR: Failed to get supported params: argument of type 'NoneType' is not iterable
2024-12-19 08:00:40,771 - 13452 - llm.py-llm:187 - ERROR: Failed to get supported params: argument of type 'NoneType' is not iterable
2024-12-19 08:00:40,780 - 13452 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=model='models/gemini-1.5-pro' google_api_key=SecretStr('**********') temperature=1.0 max_retries=2 client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022A162C5700> default_metadata=()
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers
2024-12-19 08:00:40,784 - 13452 - llm.py-llm:187 - ERROR: Failed to get supported params: argument of type 'NoneType' i


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

# Agent: PDF Content Researcher
## Task: Answer the user's question using the information from the PDF.

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

# Agent: PDF Content Researcher
## Task: Answer the user's question using the information from the PDF.

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

# Agent: PDF Content Researcher
## Task: Answer the user's question using the information from the PDF.

Provider List: https://docs.litellm.ai/docs/providers



BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=model='models/gemini-1.5-pro' google_api_key=SecretStr('**********') temperature=1.0 max_retries=2 client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022A162C5700> default_metadata=()
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [ ]:
"How did Oberyn Martel die?"